In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Disable TF debugg log

# Auxiliar imports
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer

# Model auxiliar imports
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import keras_tuner

# Model imports
from tensorflow.keras.models import load_model, Sequential  # Load saved model
from tensorflow.keras.optimizers import Adam  # Optimizer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.datasets import mnist
# from tensorflow.keras.applications import Res


KeyboardInterrupt



In [2]:
def getSetDir(set="train"):
    dirs = [f"./../dataset/archive/dataset/{set}/"]
    return dirs


# Load dataset training data
def loadData(set="train", verbose=False, size=30):
    data = []
    labels = []

    for dir in getSetDir(set):
        for subdir in os.listdir(dir):
            for img in os.listdir(dir + subdir)[0:size-1]:
                arrayLikeImage = load_img(dir + subdir + "/" + img, target_size=(300, 300))
                arrayLikeImage = img_to_array(arrayLikeImage)
                data.append(arrayLikeImage)
                labels.append(subdir)
            if verbose:
                print(f"Done with [{subdir}] images.")

    return data, labels


# trainData, trainLabels = loadData("train", size=100)
# testData, testLabels = loadData("test", size=10)
(trainData, trainLabels), (testData, testLabels) = mnist.load_data()
(trainData, trainLabels), (testData, testLabels) = (trainData/255, trainLabels), (testData/255, testLabels)


In [ ]:
trainData = np.expand_dims(trainData, -1)
testData = np.expand_dims(testData, -1)

In [ ]:
# One-Hot Encode dataset
lb = LabelBinarizer()
trainLabels = lb.fit_transform(trainLabels)
# trainLabels = to_categorical(trainLabels)  # Turn each label into a binary array

testLabels = lb.fit_transform(testLabels)
# testLabels = to_categorical(testLabels)  # Turn each label into a binary array

trainData = np.array(trainData, dtype="float32")
trainLabels = np.array(trainLabels)
testData = np.array(testData, dtype="float32")
testLabels = np.array(testLabels)

In [ ]:

# Create Model
input = Input(shape=trainData.shape[1:])
kernelWindow = (3, 3)
outputModel = Conv2D(64, kernel_size=kernelWindow[0], activation="relu", input_shape=trainData[0].shape) (input)
outputModel = MaxPooling2D(pool_size=kernelWindow) (outputModel)
outputModel = Conv2D(32, kernel_size=kernelWindow[0], activation="relu") (outputModel)
outputModel = MaxPooling2D(pool_size=kernelWindow) (outputModel)
outputModel = Flatten(name="flattten") (outputModel)
outputModel = Dense(128, activation="relu") (outputModel)
outputModel = Dropout(0.5) (outputModel)
outputModel = Dense(10, activation="softmax") (outputModel)

model = Model(inputs=input, outputs = outputModel)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 8, 32)         0         
 )                                                               
                                                                 
 flattten (Flatten)          (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 10)                20490     
                                                                 
Total params: 20,810
Trainable params: 20,810
Non-trainable params: 0
_________________________________________________________

In [ ]:
# Compile the model
STEP_LEN = 1e-6
EPOCHS = 5
BATCH_SIZE = 30
# opt = Adam(learning_rate=STEP_LEN, decay=STEP_LEN / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Train
# T = model.fit(
#     x=trainData,
#     y=trainLabels,
#     validation_data=(testData, testLabels),
#     epochs=EPOCHS)
T = model.fit(
    x=trainData,
    y=trainLabels,
    batch_size=BATCH_SIZE,
    steps_per_epoch=len(trainData) // BATCH_SIZE,
    validation_data=(testData, testLabels),
    validation_steps=len(testData) // BATCH_SIZE,
    epochs=EPOCHS)

Epoch 1/5
1875/1875 [==============================] - 110s 57ms/step - loss: 0.2558 - accuracy: 0.9283 - val_loss: 0.1060 - val_accuracy: 0.9680
Epoch 2/5
1875/1875 [==============================] - 104s 55ms/step - loss: 0.0986 - accuracy: 0.9710 - val_loss: 0.0779 - val_accuracy: 0.9754
Epoch 3/5
1875/1875 [==============================] - 125s 66ms/step - loss: 0.0752 - accuracy: 0.9776 - val_loss: 0.0632 - val_accuracy: 0.9803
Epoch 4/5
1875/1875 [==============================] - 133s 71ms/step - loss: 0.0641 - accuracy: 0.9807 - val_loss: 0.0629 - val_accuracy: 0.9794
Epoch 5/5
1875/1875 [==============================] - 102s 54ms/step - loss: 0.0557 - accuracy: 0.9834 - val_loss: 0.0525 - val_accuracy: 0.9822


In [ ]:
list(map(str, lb.classes_))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [ ]:
print("- [INFO] evaluating network...")
predictedIdxs = model.predict(testData)
predictedIdxs = np.argmax(predictedIdxs, axis=1)
print(classification_report(testLabels.argmax(axis = 1), predictedIdxs, target_names = list(map(str, lb.classes_))))

- [INFO] evaluating network...
313/313 [==============================] - 10s 31ms/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.97      0.98      1032
           3       0.99      0.98      0.99      1010
           4       0.98      0.99      0.98       982
           5       0.99      0.99      0.99       892
           6       0.99      0.99      0.99       958
           7       0.97      0.98      0.97      1028
           8       0.97      0.98      0.98       974
           9       0.98      0.96      0.97      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000

